In [32]:
# Import all the models and functions
import importlib
import models
importlib.reload(models)

<module 'models' from '/Users/surajr/classes/msse/ds200/grad-project/notebooks/models.py'>

# Example of using a model

In [21]:
# Load data
topic_and_hardness = pd.read_json(
    "../data/training-set/chatbot-arena-gpt3-scores.jsonl.gz",
    lines=True,
    compression="gzip"
)
# Perform any feature engineering
pattern_z = r"\b\w+\b"
topic_and_hardness['prompt_length'] = topic_and_hardness['prompt'].apply(lambda x: len(re.findall(pattern_z, x)))
topic_and_hardness = topic_and_hardness.dropna()
# Create a model
mlr = models.MultiLinearRegressionModel(topic_and_hardness, regularization='l1')
# Preprocess the data with the features you want to use
mlr.preprocess(features=['prompt_length'])
# Train the model
mlr.train_model()
# Evaluate the model based on a prediction
mlr.evaluate(mlr.predict())

The MSE is:  3.139633100171572


In [33]:
convos_df = pd.read_json(
    "../data/training-set/chatbot-arena-conversations.jsonl.gz",
    lines=True,
    compression="gzip"
)
# Perform any feature engineering
convos_df["prompt"] = convos_df["conversation_a"].str[0].str["content"]
convos_df['prompt_length'] = convos_df['prompt'].apply(lambda x: len(re.findall(pattern_z, x)))
convos_df = convos_df.dropna()
# Create a model
lr = models.RandomForestModel(convos_df)
# Preprocess the data with the features you want to use
lr.preprocess(features=['prompt_length'])
# Train the model
lr.train_model()
# Evaluate the model based on a prediction
lr.evaluate(lr.predict())

18.263910933739215
[[1215 1517]
 [1199 1429]]
